# PSyclone reverse mode automatic differentiation
## `psyclone.autodiff`

### 1. Differentiating some outputs of a subroutine with respect to some input variables

Let us first define two trivial Fortran subroutines.

In [1]:
source = """
subroutine bar(a, b)
    implicit none
    real, intent(in) :: a
    real, intent(out) :: b
    real :: c(10)
    integer :: i
    
    c(1) = 2
    c(2) = 4
    c(1) = 5
    c(:) = 4
    c(1:5) = 3
    b = 3
    IF (b > 4) THEN
        b = b + c(1) * a
    ELSE
        b = b**2 + a
    END IF

    do i = 1, 11, 2
        b = c**i
    end do

end subroutine bar

subroutine foo(x,w,f,g)
    implicit none
    real, intent(in) :: x, w
    real, intent(out) :: f
    real, intent(out) :: g

    f = w**2
    g = 3 * x + cos(x)
    IF (g > 4) THEN
        g = g + x
    ELSE
        g = g - x
    END IF
    
    call bar(x+2*f, f)
end subroutine foo
"""

To use PSyclone's `autodiff` module, 
- first create a PSyIR AST from the source, 
- walk it to get the `[File]Container` node containing the function of interest (here `foo`),
- instantiate an `ADReverseContainerTrans`.

In [2]:
from psyclone.autodiff.transformations import ADReverseContainerTrans
from psyclone.psyir.frontend.fortran import FortranReader
from psyclone.psyir.backend.fortran import FortranWriter
from psyclone.psyir.nodes import Container
from psyclone.autodiff import ADJointReversalSchedule, ADSplitReversalSchedule, ADLinkReversalSchedule

# Front- and backend
freader = FortranReader()
fwriter = FortranWriter()

# PSyIR
psy = freader.psyir_from_source(source)

# Get the container
container = psy.walk(Container)[0]

# Initialize the container transformation
container_trans = ADReverseContainerTrans()

Now apply the `ADReverseContainerTrans`` instance, whose arguments are:
- the name of the routine to differentiate,
- a list of names of dependent variables, to differentiate (numerators in derivatives),
- a list of names of independent variables, to differentiate *with respect to* (denominators in derivatives),
- an `ADReversalSchedule` subclass instance, specifying how routines call one another,
- some options.

Then use a backend to write the result of the transformation back to Fortan.

In [3]:
# What to differentiate and wrt to what
# d{dependent}/d{independent}
routine_name = 'foo'
dependent_vars = ['f','g']
independent_vars = ['x','w']

# Reversal schedules that are available at this date
split_schedule = ADSplitReversalSchedule()
joint_schedule = ADJointReversalSchedule()
link_schedule = ADLinkReversalSchedule( strong_links=[['foo', 'bar']],
                                        #weak_links=[['foo', 'bar']],
                                        default_link='weak')
reversal_schedule = split_schedule

# Available options, for now
options = {'verbose': True,                     # default is False
           'jacobian': True,                    # default is False
           'simplify': True,                    # default is True
           'simplify_times': 5,                 # default is 5
           'inline_operation_adjoints': True}   # default is True

# Apply the transformation
result = container_trans.apply(container, 
                               routine_name, 
                               dependent_vars, 
                               independent_vars, 
                               reversal_schedule, 
                               options=options)

# Transformed source
print(fwriter(result))

VisitorError: Visitor Error: The following symbols are not explicitly declared or imported from a module and there are no wildcard imports which could be bringing them into scope: 'i'

### Using the transformed routines
To obtain the differential of `f` or `g` as returned by `foo` with respect to `x` or `w`, *ie* to obtain $\dfrac{\partial \{f,g\}}{\partial \{x, w\}}(x, w)$ call `foo_rev(x, x_adj, w, w_adj, f, f_adj, g, g_adj)` with arguments:
- `x` and `w` the point to differentiate in,
- `x_adj = 0.0, w_adj = 0.0`, these being the adjoints of the dependent variables, which will accumulate derivatives,
- one of **either**
    - `f_adj = 1.0, g_adj = 0.0` to obtain `x_adj` = $\dfrac{\partial f}{\partial x}(x, w)$ and `w_adj` = $\dfrac{\partial f}{\partial w}(x, w)$,
    - `f_adj = 0.0, g_adj = 1.0` to obtain `x_adj` = $\dfrac{\partial g}{\partial x}(x, w)$ and `w_adj` = $\dfrac{\partial g}{\partial w}(x, w)$.

### Jacobian routine
A more convenient `foo_jacobian(f, g, x, w)` subroutine is also generated, due to the option `'jacobian' = True`.
It can be called without bothering about defining initials values of the adjoints arguments to obtain the Jacobian of $\begin{pmatrix} f \\ g \end{pmatrix}$ with respect to $\begin{pmatrix} x \\ w \end{pmatrix}$ since the variables are all scalar valued.

### Reversal schedules
The reversal used here was an `ADSplitReversalSchedule`, meaning that when the *recording routine* `foo_rec` is called, recording values to the tape, it calls `bar_rec`, which records as well, and the same for the *returning routins* `foo_ret` and `bar_ret`.
To achieve this, the corresponding slice of the tape of `foo` is passed as argument to `bar_rec` and `bar_ret`.

Switching to an `ADJointReversalSchedule`, `foo_rec` calls the original (or advancing) `bar`, while `foo_ret` calls the *reversing routine* `bar_rev`, itself calling its recording and returning routines.
In that case, the tape of `bar` is local to `bar_rev` and independent from the tape of `foo`, its values being uneeded after `bar_rev` returns, so no slice is passed.
    
### Transformation of called routines
Since `foo` contains a call to `bar`, note that `autodiff` automatically differentiates the called subroutine.
In the absence at the moment of activity analysis, it assumes for arguments of the called subroutine:
- all `intent(out)`, `intent(inout)` and `unspecified` intent variables to be independent,
- all `intent(in)`, `intent(inout)` and `unspecified` intent variables to be dependent.

### Taping
Note that non-returned values that may appear in some adjoint computations (*eg* the second value of `c` in `bar`) are taped before the recording routine returns.

## Limitations and ideas:

### Vectors and matrices
For now `psyclone.autodiff` only supports real, scalar variables.
As it is, even `ArrayReference` nodes with `Literal` shape dimensions will raise errors, mostly because creating their adjoint variables is not supported yet.

### Containers
For now the definitions of called routines should all be in the `container` node argument of the `ADReverseContainerTrans`, or a descendant node of it.
The transformed routines are all added to that `container`, independently of where they were found.

### Taping
Due to the absence of activity and TBR analyses, note that some values (*eg* the first of `c` in `bar`) are not used afterwards and yet taped.

It would probably be easier to store every value in a single 1 dimensional array, by inserting vectors and matrices after flattening them, than try to use a tape containing elements of different dimensions.

Depending on the diversity of types (mostly real values but maybe sometimes a few integers?) and kinds, it might make sense to have multiple tapes or a single one accepting these different types and kinds, if that's doable in Fortan and PSyclone (pointers? allocate attributes in a derived type serving as element of the tape?).

### Adjoint datatype
For now all adjoints default to `real`.

For vector variables we'd need to determine the shape. 
In particular for intermediate operations datatypes, see below.

### Operations results
For now non assigned operation results are not taped but recomputed, even when they appear in subsequent adjoint computations.
Implementing the datatypes of `psyclone.psyir.nodes.[Unary, Binary, Nary]Operation` nodes might be useful for this. Types and kinds seem to follow fairly simple rules (except in a few cases?). 
Shapes of vectors and matrices would be needed to tape vector operations results and deduce the shapes of adjoints when extending to vector-valued variables.

### For loops, control flow, functions, programs
None of these were implement yet.
- Control flow, functions (and programs, if useful at all) should be pretty easy.
    - For **control flow**, either record the boolean values of conditions or tag (index) the conditional branches and record which were taken.
    - **Functions** will need to have adjoints defined, same as done for operations (`op_adj` variables).
    - Pass the its (result) adjoint to the transformed fonction as an argument.
    - Either pass the tape and have the function modify it (not very Fortran-minded?) or transform the function to a subroutine?
- `do` loops with split reversal schedules may require a run-time index offset to either pass the correct slice `(i:)` of the caller routine, or to know the first index to use in the called routine. In any case, the called recording routine should probably return the number of tape elements it wrote.

### Activity analysis
This is essentially dependence DAG related. Note that this would be useful for the transformation of called routines (see the previous cell for the way it's currently done).

### TBR (to be recorded) analysis
Using the dependence DAG, determine if the values of `Reference` nodes appear as arguments of subsequent active `Call` or **non-linear** `Operation` nodes (on the differentiated side or the other, depending on the operator. Consider *eg* `x*y` and `x**2`.).
For the result values of `Operation` nodes, it may not always make sense to tape them at all, depending on how many there are.

### Taping vs recomputing
Recomputing and checkpointing strategies should ideally be implemented. 
Their efficiency clearly depends on the activity analysis, there is no point evaluating the whole routine again for a single value amongst many when it can be helped.